In [ ]:
%load_ext watermark
%watermark  -d -u -a 'Andreas Mueller, Kyle Kastner, Sebastian Raschka' -v -p numpy,scipy,matplotlib,scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 无监督学习(Unsupervised Learning) Part 2 -- 聚类(Clustering)

聚类是将样本组织到具有相似性分组的任务，本节中，将探讨一个基本的聚类任务，针对一些合成的和真实世界的数据集。

以下是聚类算法的一些常见应用:

- 数据缩减和压缩
- 数据摘要，作为推荐系统的再处理步骤
- 类似的:
    - 将相关的网上新闻(如Google新闻)和网上搜寻结果分组
    - 为投资组合管理分组相关股票报价
    - 建立客户档案，以便进行市场分析
- 为无监督特征提取建立一个原型样本码本

从创建一个简单的二维合成数据集开始:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=42)
X.shape

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);

上面的散点图中，可以看到三组独立的数据点，我们希望用聚类来恢复它们——想象一下在分类任务中“发现”我们已经认为理所当然的类标签。

即使这些群组在数据中是显而易见的，但当数据存在于高维空间时，也很难找到它们，因为我们无法用单独的直方图或散点图进行可视化。

先用最简单的聚类算法之一，K均值算法。用迭代算法搜索3个群组的中心，使得每个点到所属类的距离最小化。K均值的标准实现用的是欧式距离，这就是为什么我们希望确保所有变量都在同一尺度上进行测量，如果使用真实世界的数据。在前面，已经讨论了实现这一目标的一种技术，即标准化。

**问题**: 你希望输出结果是什么样的？

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)

可以通过调用fit，然后访问K均值估计器的``labels_``属性，或通过调用``fit_predict``。无论哪种方式，结果都包含分配给每个点的类的标识。

In [ ]:
labels = kmeans.fit_predict(X)

In [ ]:
labels

In [ ]:
all(y == labels)

把结果可视化：

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels);

与真正的标签相比:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y);

在这里，我们可能对聚类结果感到满意。但总的来说，我们会希望有一个更量化的评价。在生成blob时，将我们的聚类标签与我们的基本事实进行比较怎么样？

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score:', accuracy_score(y, labels))
print(confusion_matrix(y, labels))

In [ ]:
np.mean(y == labels)

---

**练习**

在看了“真”标签阵列y和上面的散点图和`labels`后，你能理解为什么我们的计算精度是0.0，而不是1.0，你能修正它吗？

---

尽管我们完美地恢复了数据到类的划分，但是我们分配的类标识是任意的，我们不能指望完全恢复它们。因此，我们必须采用不同的评分标准，如``adjusted_rand_score``，它对标签排列是不变的:

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, labels)

K-means的“缺点”之一，是我们必须指定聚类的个数，而我们通常不知道*先验(apriori)*。例如，让我们看看如果将合成的3-blob数据集中的聚类个数设置为2会发生什么:

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels);

#### 手肘(Elbow)法

手肘法是一种“经验”方法，用于找到最佳的聚类个数。这里，我们看一下不同K值的类内分散度:

In [ ]:
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

plt.show()

然后，选择类似“肘部”位置的值。如我们所见，在这种情况下，k=3，考虑到我们之前对数据集的视觉预期，这是有意义的。

**聚类伴随假设而来**: 聚类算法通过假设样本应该被分组在一起，来发现聚类。每种算法都有不同的假设，结果的质量和可解释性将取决于这些假设是否满足目标。对于K-均值聚类，模型假设所有的聚类都有相等的球形方差。

**一般来说，不能保证聚类算法找到的结构与你感兴趣的东西有任何关系**.


我们可以很容易创建一个具有非各向同性聚类的数据集，在该数据集上K均值会失败:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

y_pred = KMeans(n_clusters=3).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred)

## 一些值得注意的聚类方法

以下是一些众所周知的聚类算法。

- `sklearn.cluster.KMeans`: <br/>
    最简单有效的聚类算法。需要预先提供计算聚类个数，假设数据已经标准化
- `sklearn.cluster.MeanShift`: <br/>
    可以找到比KMeans看起来更好看的聚类，但无法扩展到大量样本
- `sklearn.cluster.DBSCAN`: <br/>
    可以基于密度检测不规则形状的类簇，即输入空间中的稀疏区域可能成为簇间边界。也可以检测离群值(不属于聚类的样本)
- `sklearn.cluster.AffinityPropagation`: <br/>
    基于数据点间消息传递的聚类算法
- `sklearn.cluster.SpectralClustering`: <br/>
    应用于归一化图拉普拉斯投影的K均值
- `sklearn.cluster.Ward`: <br/>
    基于沃德算法实现分层聚类，方差最小化方法。在每一步，最小化的总和所有聚类内平方差(惯性标准)

其中，Ward, SpectralClustering, DBSCAN和Affinity propagation也可用预先计算的相似性矩阵进行处理

<img src="figures/cluster_comparison.png" width="900">

## 练习: digits聚类

对手写数字数据进行K均值聚类，找到10个聚类。将类中心可视化为图像(将每个样本reshape到8×8并使用``plt.imshow``)聚类和特定数字有关吗？``adjusted_rand_score``得分是多少？

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
# ...

In [ ]:
#%load solutions/08B_digits_clustering.py